In [1]:
!pip install openpyxl pandas

In [2]:
import pandas as pd
import openpyxl

In [3]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/capstone/shuffled_products.csv')
data.head()

Mounted at /content/drive


,title,normal_price,label,image_url
0,Cedea Steamboat Set 500G,30%,makanan,https://assets.klikindomaret.com/products/2010...
1,Charm Pembalut Safe Night 7'S Wing 42cm,23%,kesehatan,https://assets.klikindomaret.com/products/2001...
2,Twinings Green Tea Jasmine 10's,Rp 42.700,minuman,https://assets.klikindomaret.com/products/2013...
3,Burung Dara Mie Telur Urai Original 140G,Rp 5.600,makanan,https://assets.klikindomaret.com/products/2008...
4,Indomaret Keripik Bothe Talas Original 50G,Rp 8.200,makanan,https://assets.klikindomaret.com/products/2013...


In [4]:
from openpyxl import Workbook

In [5]:
import random
from datetime import datetime, timedelta

def generate_random_datetime():
    # Start and end range for dates
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2024, 11, 27)
    # Generate a random date and time
    random_date = start_date + timedelta(
        seconds=random.randint(0, int((end_date - start_date).total_seconds()))
    )
    return random_date.strftime("%d.%m.%y-%H:%M")


In [6]:
def generate_random_products():
  products = {}
  number_of_products = random.randint(1,16)
  sampled_titles = data['title'].sample(number_of_products).str.upper()  # Randomly sample product titles

  for title in sampled_titles:
    random_price = random.randint(10, 900) * 100
    products[title] = random_price

  return products

In [7]:
from openpyxl.styles import Border, Side

def apply_outside_border(ws):
    # Define border styles
    thin_border = Side(border_style="thin", color="000000")

    # Get the dimensions of the worksheet
    min_row, max_row = 1, ws.max_row
    min_col, max_col = 1, ws.max_column

    # Apply border to the first and last row
    for col in range(min_col, max_col + 1):
        # Top row
        cell = ws.cell(row=min_row, column=col)
        cell.border = Border(
            top=thin_border,
            left=cell.border.left,
            right=cell.border.right,
            bottom=cell.border.bottom,
        )
        # Bottom row
        cell = ws.cell(row=max_row, column=col)
        cell.border = Border(
            bottom=thin_border,
            left=cell.border.left,
            right=cell.border.right,
            top=cell.border.top,
        )

    # Apply border to the first and last column
    for row in range(min_row, max_row + 1):
        # Left column
        cell = ws.cell(row=row, column=min_col)
        cell.border = Border(
            left=thin_border,
            top=cell.border.top,
            bottom=cell.border.bottom,
            right=cell.border.right,
        )
        # Right column
        cell = ws.cell(row=row, column=max_col)
        cell.border = Border(
            right=thin_border,
            top=cell.border.top,
            bottom=cell.border.bottom,
            left=cell.border.left,
        )


In [8]:
def apply_print_area(ws):
    min_row, max_row = 1, ws.max_row
    ws.print_area = f"A{min_row}:D{max_row}"


In [9]:
from openpyxl.styles import Alignment, Border, Side
from openpyxl.utils import get_column_letter

def create_receipts(worksheet_name, wb):
    # Create a new worksheet or get the existing one
    if worksheet_name in wb.sheetnames:
        ws = wb[worksheet_name]
    else:
        ws = wb.create_sheet(title=worksheet_name)

    # nambahin kop
    ws['A1'] = "PT. INDOMARET PRISMATAMA"
    ws['A2'] = "JL ANCOL U9, ANCOL"
    ws['A3'] = "BARAT- JAKARTA UTARA"
    ws['A4'] = "NPWP 123456789"

    # alamat indomaret
    ws['A6'] = "KS TUBUN 0293493733"
    ws['A7'] = "JL KS TUBUN NO.05 RTO2 RWO4 KEL TEMANGGUNG"
    ws['A8'] = "KEC TEMANGGUNG KAB TEMANGGUNG, 56213"
    for i in range(3):
      ws.merge_cells(start_row=6+i, start_column=1, end_row=6+i, end_column=4)
      ws['A'+str(6+i)].alignment = Alignment(horizontal='center')

    # informasi bon
    ws['A10'] = generate_random_datetime()
    ws['B10'] = "2.1.27"
    content = ws['B10'].value
    ws.column_dimensions['B'].width = len(content)
    ws['C10'] = "301135/RATIH/01"
    ws.merge_cells(start_row=10, start_column=3, end_row=10, end_column=4)

    # informasi product
    products = generate_random_products()
    for i, (product_name, product_price) in enumerate(products.items()):
      random_amount_item = random.randint(1, 8)
      ws.cell(row=12 + i, column=1, value=product_name)
      ws.cell(row=12 + i, column=2, value=random_amount_item)
      ws.cell(row=12 + i, column=3, value=product_price)
      ws.cell(row=12 + i, column=4, value=product_price * random_amount_item)

    max_length = 36
    ws.column_dimensions[get_column_letter(1)].width = max_length

    # informasi uang
    start_row_info = 12 + len(products)
    total_cost = sum(product_price * random_amount_item for product_price in products.values())
    random_cash = total_cost + (random.randint(0, 100) * 1000)

    # Isi data dan lakukan merge
    for i, (label, value) in enumerate([
      ("HARGA JUAL", total_cost),
      ("TUNAI", random_cash),
      ("KEMBALI", random_cash - total_cost)
      ]):
      ws['B' + str(start_row_info + i)] = label  # Isi label di kolom B (sel paling kiri)
      ws['B' + str(start_row_info + i)].alignment = Alignment(horizontal='right')  # Rata kanan untuk teks
      ws['D' + str(start_row_info + i)] = value  # Isi nilai di kolom D

      # Merge kolom B dan C
      ws.merge_cells(start_row=start_row_info + i, start_column=2, end_row=start_row_info + i, end_column=3)


    # footer nota
    start_row_footer = start_row_info + 4
    ws['A' + str(start_row_footer)] = "PPN : DPP = 8,919 PPN = 981"
    ws['A' + str(start_row_footer+1)] = "NON PPN : DPP = 100"
    ws['A' + str(start_row_footer+2)] = "LAYANAN KONSUMEN"
    ws['A' + str(start_row_footer+3)] = "SMS/WA 0811.1500.280 TELP 1500280"
    ws['A' + str(start_row_footer+4)] = "KONTAK@INDOMARET.CO.ID"
    ws['A' + str(start_row_footer+5)] = "BELANJA LEBIH MUDAH DI KLIKINDOMARET"
    ws['A' + str(start_row_footer+6)] = "GRATIS ONGKIR 1 JAM SAMPAI"
    for i in range(7):
      ws.merge_cells(start_row=start_row_footer+i, start_column=1, end_row=start_row_footer+i, end_column=4)
      if i > 1:
        ws['A'+str(start_row_footer+i)].alignment = Alignment(horizontal='center')

    # bikin border
    apply_outside_border(ws)

    # bikin print area
    apply_print_area(ws)



    return ws


In [10]:
# Create a new workbook
wb = Workbook()

# Dictionary to store worksheets
worksheets = {}

for i in range(10):
    sheet_name = f"sheet{i+1}"
    worksheets[sheet_name] = wb.create_sheet(title=sheet_name)
    create_receipts(sheet_name, wb)

# Save the workbook to a file
wb.save("nota.xlsx")
# Print a success message
print("Excel file created successfully!")

Excel file created successfully!
